In [113]:
from webtableparser import WebTableParser
import pandas as pd
# import numpy as np
# import re

site = WebTableParser()
site.create('https://www.fundsexplorer.com.br/ranking', 'table table-hover')
table = site.capture()
df = site.parse(table)


In [114]:
# Dataframe manipulation should be done according with the table
df.columns = ['codigo', 'setor', 'precoatualR$', 'liqdiariaNeg', 'dividR$', 'divyield%', 'dy3macum%', 'dy6macum%', 'dy12macum%', 'dy3mmedia%', 'dy6mmedia%', 'dy12mmedia%', 'dyano%',  'varpreco%', 'rentper%', 'rentacum%', 'patrliqR$', 'vpaR$', 'p/vpaN', 'dypatr%', 'varpatr%', 'rentpatrper%', 'rentpatracum%', 'vacfisica%', 'vacfinan%', 'qtdativosN']
df = df.applymap(lambda x: str(x).replace('R$', ''))
df = df.applymap(lambda x: str(x).replace('%', ''))
df['precoatualR$'] = df['precoatualR$'].apply(lambda x: str(x).replace('.', ''))
df['patrliqR$'] = df['patrliqR$'].apply(lambda x: str(x).replace('.', ''))
df['vpaR$'] = df['vpaR$'].apply(lambda x: str(x).replace('.', ''))
df = df.applymap(lambda x: str(x).replace(',', '.'))
df['setor'] = df['setor'].apply(lambda x: str(x).replace('Ã', 'i'))
# df['setor'] = df['setor'].apply(lambda x: re.sub(r'Ã ', 'i', x)) #alternative using regex (import re needed)

In [115]:
df['codigo'] = df['codigo'].astype('string')
df['setor'] = df['setor'].astype('string')
df['precoatualR$'] = pd.to_numeric(df['precoatualR$'], errors='coerce')
df['liqdiariaNeg'] = pd.to_numeric(df['liqdiariaNeg'], errors='coerce')
df['dividR$'] = pd.to_numeric(df['dividR$'], errors='coerce')
df['divyield%'] = pd.to_numeric(df['divyield%'], errors='coerce')
df['dy3macum%'] = pd.to_numeric(df['dy3macum%'], errors='coerce')
df['dy6macum%'] = pd.to_numeric(df['dy6macum%'], errors='coerce')
df['dy12macum%'] = pd.to_numeric(df['dy12macum%'], errors='coerce')
df['dy3mmedia%'] = pd.to_numeric(df['dy3mmedia%'], errors='coerce')
df['dy6mmedia%'] = pd.to_numeric(df['dy6mmedia%'], errors='coerce')
df['dy12mmedia%'] = pd.to_numeric(df['dy12mmedia%'], errors='coerce')
df['dyano%'] = pd.to_numeric(df['dyano%'], errors='coerce')
df['varpreco%'] = pd.to_numeric(df['varpreco%'], errors='coerce')
df['rentper%'] = pd.to_numeric(df['rentper%'], errors='coerce')
df['rentacum%'] = pd.to_numeric(df['rentacum%'], errors='coerce')
df['patrliqR$'] = pd.to_numeric(df['patrliqR$'], errors='coerce')
df['vpaR$'] = pd.to_numeric(df['vpaR$'], errors='coerce')
df['p/vpaN'] = pd.to_numeric(df['p/vpaN'], errors='coerce')
df['dypatr%'] = pd.to_numeric(df['dypatr%'], errors='coerce')
df['varpatr%'] = pd.to_numeric(df['varpatr%'], errors='coerce')
df['rentpatrper%'] = pd.to_numeric(df['rentpatrper%'], errors='coerce')
df['rentpatracum%'] = pd.to_numeric(df['rentpatracum%'], errors='coerce')
df['vacfisica%'] = pd.to_numeric(df['vacfisica%'], errors='coerce')
df['vacfinan%'] = pd.to_numeric(df['vacfinan%'], errors='coerce')
df['qtdativosN'] = pd.to_numeric(df['qtdativosN'], errors='coerce')
df = df.fillna(0)  # all NaNs filled with zero
# df['liqdiariaNeg'] = df['liqdiariaNeg'].fillna(0)  # column by column if needed
# df['liqdiariaNeg'] = df['liqdiariaNeg'].replace(np.nan, 0, regex=True)  # column by column with regex
df['liqdiariaNeg'] = df['liqdiariaNeg'].astype('int64')
df['qtdativosN'] = df['qtdativosN'].astype('int64')

In [116]:
print(df)
print(df.info())

codigo                 setor  precoatualR$  liqdiariaNeg  dividR$  \
0    FIVN11             Shoppings          4.44           218     0.00   
1    BZLI11  Ti­tulos e Val. Mob.         15.29         48432     0.00   
2    XTED11    Lajes Corporativas          7.30           572     0.00   
3    ALMI11    Lajes Corporativas       1180.00             9     0.00   
4    DOMC11    Lajes Corporativas        460.00           623     0.00   
..      ...                   ...           ...           ...      ...   
177  FCFL11                Outros        112.60          2113     0.57   
178  XPIN11                Outros        114.76         32008     0.62   
179  XPML11             Shoppings        109.50         28794     0.18   
180  FLRP11             Shoppings       1398.97            48     1.74   
181  ALZR11            Logi­stica        138.51          9575     0.57   

     divyield%  dy3macum%  dy6macum%  dy12macum%  dy3mmedia%  ...  \
0         0.00       0.00       0.00        0.0